In [ ]:
pip install gradio transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.6 MB/s eta 0:00:00


In [ ]:
import gradio as gr
import torch
import numpy as np
from transformers import pipeline
import requests
from huggingface_hub import login

# Hugging Face API details
API_KEY = "****API KEY***"
API_URL = "https://api-inference.huggingface.co/models/tiiuae/falcon-7b-instruct"

# Authenticate Hugging Face token
login(token=API_KEY)

# Function to generate AI response for food recommendation
def generate_ai_response(text, sentiment, category):
    print("\n🔹 Generating AI Response...")
    print(f"Text: {text}")
    print(f"Sentiment: {sentiment}")
    print(f"Category: {category}")

    prompt = (f"The input is a mood: '{text}'.\n"
              f"It expresses a {sentiment.lower()} sentiment and belongs to the '{category}' category.\n"
              f"Suggest a unique and detailed food recipe that aligns with this mood, focusing on ingredients and preparation steps.")

    headers = {"Authorization": f"Bearer {API_KEY}"}
    payload = {"inputs": prompt, "parameters": {"max_new_tokens": 300, "temperature": 0.7, "top_p": 0.9, "repetition_penalty": 1.2}}

    try:
        response = requests.post(API_URL, headers=headers, json=payload)
        print(f"API Response Code: {response.status_code}")

        if response.status_code == 200:
            generated_text = response.json()[0]["generated_text"][len(prompt):].strip()
            print(f"Generated AI Response: {generated_text}")
            return generated_text
        else:
            print(f"❌ API Error: {response.json()}")
            return f"❌ Error: {response.json()}"
    except Exception as e:
        print(f"Exception during API call: {e}")
        return "❌ API call failed"

# Load models
print("🔹 Loading models...")

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

sentiment_pipeline = pipeline("sentiment-analysis", model="distilbert/distilbert-base-uncased-finetuned-sst-2-english")
print("✅ Sentiment Analysis Model loaded successfully!")

# Define food categories
positive_food_categories = ["energizing", "light", "healthy", "refreshing", "comforting"]
negative_food_categories = ["comfort food", "heavy", "spicy", "cheerful"]
unhealthy_food_categories = ["unhealthy", "fried", "sugary", "junk", "greasy"]
sick_food_categories = ["sick", "easy-to-digest", "light", "comfort food", "warm"]

keyword_to_food_category = {
    "light": "light", "refreshing": "refreshing", "healthy": "healthy", "energizing": "energizing",
    "comfort": "comforting", "heavy": "heavy", "spicy": "spicy", "cheerful": "cheerful",
    "unhealthy": "unhealthy", "fried": "unhealthy", "sugary": "unhealthy", "junk": "unhealthy",
    "greasy": "unhealthy", "sick": "sick", "digest": "easy-to-digest", "warm": "comfort food"
}

# Function to recommend food based on mood
def recommend_food(mood_text):
    print("\n🔹 Processing Mood for Food Recommendation...")

    # Sentiment analysis
    sentiment_result = sentiment_pipeline(mood_text)
    sentiment_label = sentiment_result[0]['label']
    print(f"🔹 Sentiment: {sentiment_label}")

    # Choose food categories based on sentiment
    selected_food_categories = positive_food_categories if sentiment_label == "POSITIVE" else negative_food_categories
    print(f"🔹 Selected Food Categories: {selected_food_categories}")

    # Priority category based on text keywords
    priority_category = None
    for word, category in keyword_to_food_category.items():
        if word in mood_text.lower():
            priority_category = category
            break
    print(f"🔹 Priority Category: {priority_category}")

    # Select final category
    if "sick" in mood_text.lower():
        selected_food_categories = sick_food_categories
    elif "unhealthy" in mood_text.lower():
        selected_food_categories = unhealthy_food_categories
    final_category = priority_category if priority_category else selected_food_categories[0]
    confidence_score = 1.0 if priority_category else 0.9  # Give higher confidence to priority category

    print(f"✅ Final Category: {final_category} (Confidence: {confidence_score:.4f})")

    # Generate AI food response
    ai_response = generate_ai_response(mood_text, sentiment_label, final_category)

    return ai_response  # Only return AI-generated response

# Gradio Interface with updated UI
print("🔹 Initializing Gradio Interface...")
iface = gr.Interface(
    fn=recommend_food,
    inputs=gr.Textbox(label="Enter Your Mood (Text)", lines=2, placeholder="Enter mood here..."),
    outputs=gr.Textbox(label="AI-Generated Food Recommendation", lines=8, placeholder="Food recommendation will appear here..."),
    title="Mood-Based Food Recommendation",
    description="Enter a mood, and AI will generate a unique food recommendation based on your mood and sentiment.",
      # Use Hugging Face UI theme
     # Vertical layout for easy reading
    allow_flagging="never",  # Disable flagging
    live=True,  # Enable live updates
    css="""
    body {background-color: #f8f5e1; font-family: 'Arial', sans-serif;}
    .gradio-container {border-radius: 15px; border: 2px solid #e7b748; padding: 20px;}
    .gr-button {background-color: #f29c42; color: white; border-radius: 10px; padding: 15px;}
    .gr-button:hover {background-color: #f1b850;}
    .gradio-input {background-color: #fff7e6; border: 1px solid #e7b748; padding: 15px; border-radius: 8px;}
    .gradio-output {background-color: #fff7e6; border: 1px solid #e7b748; padding: 20px; border-radius: 8px;}
    .gradio-container img {max-height: 100px; margin-bottom: 10px;}
    """,  # Custom CSS for color and UI improvements
)

# Launch Gradio
print("🚀 Launching Gradio App...")
iface.launch(debug=True)

🔹 Loading models...
Using device: cpu


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cpu
/usr/local/lib/python3.11/dist-packages/gradio/interface.py:415: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


✅ Sentiment Analysis Model loaded successfully!
🔹 Initializing Gradio Interface...
🚀 Launching Gradio App...
Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://68f3581e8f928e1c40.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)



🔹 Processing Mood for Food Recommendation...
🔹 Sentiment: POSITIVE
🔹 Selected Food Categories: ['energizing', 'light', 'healthy', 'refreshing', 'comforting']
🔹 Priority Category: None
✅ Final Category: energizing (Confidence: 0.9000)

🔹 Generating AI Response...
Text: S
Sentiment: POSITIVE
Category: energizing
API Response Code: 200
Generated AI Response: A bright and energizing breakfast idea: 'Sunrise Citrus Smoothie Bowl'.
Ingredients: 1 banana, 1 orange, 1/2 cup frozen mixed berries, 1/2 cup almond milk, 1 scoop protein powder, 1 teaspoon honey, 1/4 teaspoon ground cinnamon, 1/2 teaspoon vanilla extract, 1/4 teaspoon nutmeg, and 1/2 cup rolled oats.

Instructions: 1. Place all ingredients in a blender and blend until smooth.
2. Pour into two separate bowls.
3. In one bowl, add a few tablespoons of rolled oats for extra texture.
4. Sprinkle each smoothie bowl with a pinch of cinnamon and nutmeg on top for added flavor.
5. Enjoy your energizing 'Sunrise Citrus Smoothie Bowls'!

🔹 Pr